In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

In [4]:
data = pd.read_csv('../CDS-LM/data/finetuning/saluki/human_sss_reprocessed.csv', index_col=0)

In [10]:
data['CDSLen'] = data['CDS'].apply(len)
data['LenBin'] = pd.qcut(data['CDSLen'], 5, labels=[0,1,2,3,4])

In [23]:
data.set_index('Gene', inplace=True)

In [12]:
saluki_preds = pd.read_csv('saluki_preds.csv', index_col=0)
saluki_preds

,gene,preds,targets,fold,cross
0,ENSG00000000457,0.16850,-1.437500,0,0
1,ENSG00000001631,0.08484,-0.184937,0,0
2,ENSG00000002549,1.18600,1.395508,0,0
3,ENSG00000003137,0.71500,-0.657227,0,0
4,ENSG00000003987,0.37800,0.064392,0,0
...,...,...,...,...,...
64835,ENSG00000273045,-0.53560,-0.976074,9,4
64836,ENSG00000274286,-0.70850,-0.480713,9,4
64837,ENSG00000275004,-2.14000,-1.830078,9,4
64838,ENSG00000275183,-1.35400,-1.360352,9,4


In [13]:
cdslm_preds = pd.read_csv('CDS-LM-2048/test_preds.csv', index_col=0)
cdslm_preds

,pred,y_true
0,0.652344,0.090027
1,0.300781,0.493896
2,0.597656,1.508789
3,-0.660156,-0.536133
4,-0.500000,-1.348633
...,...,...
12909,-1.757812,-1.743164
12910,-1.304688,-1.372070
12911,-1.898438,-2.175781
12912,0.213867,-1.517578


In [14]:
cdslm_preds['LenBin'] = data['LenBin']

In [ ]:
for lenbin in range(5):
    subset = cdslm_preds[cdslm_preds['LenBin'] == lenbin]
    print(spearmanr(subset['pred'], subset['y_true']))

In [41]:
cdslm_preds.to_csv('CDS-LM_preds.csv')

In [39]:
ens_preds

,gene,preds,targets,LenBin
0,ENSG00000000003,1.366320,1.793945,0.0
1,ENSG00000000457,-0.166648,-1.437500,3.0
2,ENSG00000000460,0.406774,0.090027,4.0
3,ENSG00000000938,1.303200,-0.197021,2.0
4,ENSG00000000971,-0.009372,1.599609,4.0
...,...,...,...,...
12963,ENSG00000278615,0.639960,0.260498,0.0
12964,ENSG00000278619,-0.508720,-0.278076,1.0
12965,ENSG00000278845,0.274960,0.384766,1.0
12966,ENSG00000280789,-1.413600,-0.500977,0.0


In [40]:
ens_preds.to_csv('saluki_ensemble_preds.csv')

In [33]:
ens_preds = saluki_preds.groupby('gene')[['preds', 'targets']].mean().reset_index()

In [34]:
gene_bin_dict = data['LenBin'].to_dict()
ens_preds['LenBin'] = [gene_bin_dict.get(gene) for gene in ens_preds['gene']]

In [35]:
ens_preds.dropna(inplace=True)

In [36]:
for lenbin in range(5):
    subset = ens_preds[ens_preds['LenBin'] == lenbin]
    print(spearmanr(subset['preds'], subset['targets']))

SignificanceResult(statistic=np.float64(0.7316436821350907), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7549508023423297), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7691705244372246), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7667858996601215), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7424743594684269), pvalue=np.float64(0.0))


In [37]:
for lenbin in range(5):
    subset = cdslm_preds[cdslm_preds['LenBin'] == lenbin]
    print(spearmanr(subset['pred'], subset['y_true']))

SignificanceResult(statistic=np.float64(0.7591170934498667), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7459861129868056), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7563678694435764), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7406230053904052), pvalue=np.float64(0.0))
SignificanceResult(statistic=np.float64(0.7482090578087189), pvalue=np.float64(0.0))


In [ ]:
# Now repeat with Saluki/CDS-LM combined.
# Bootstrap same number with replacement from each bin and calculate Spearman's correlation for 100(0) replicates (confidence ints.)
# Compare proportion of time CDS-LM > Saluki for each bin and vice versa.